In [4]:
!pip install -q git+https://github.com/huggingface/transformers.git bitsandbytes #0.41.0

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.5 MB/s eta 0:00:00


In [2]:
pip install accelerate==0.21.0

  Using cached accelerate-0.21.0-py3-none-any.whl (244 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.22.0.dev0
    Uninstalling accelerate-0.22.0.dev0:
      Successfully uninstalled accelerate-0.22.0.dev0


In [5]:
import accelerate
import bitsandbytes

In [6]:
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
import torch

processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")
model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b", load_in_4bit=True, torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
#Artimuse interface demo through Gradio
!pip install gradio
import os
import gradio as gr

from PIL import Image
import requests
def set_image(img):
    return img

def format_chat_prompt(message, chat_history):
    prompt = "" #instantiate variable
    for turn in chat_history: #append past history
        user_message, bot_message = turn
        prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
    prompt = f"{prompt}\nUser: {message}\nAssistant:" #add current prompt
    return prompt

def respond(image, message, chat_history):
        formatted_prompt = format_chat_prompt(message, chat_history)
        inputs = processor(images=image, text=formatted_prompt, return_tensors="pt").to(device="cuda", dtype=torch.float16)
        # autoregressively generate an answer
        outputs = model.generate(
                **inputs,
                num_beams=10,
                max_new_tokens=100,
                min_length=5,
                repetition_penalty=1.5,
                length_penalty=1.0,
                temperature=1,
        )
        outputs[outputs == 0] = 2 # this line can be removed once https://github.com/huggingface/transformers/pull/24492 is fixed
        generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
        print(generated_text)
        chat_history.append((message, generated_text))
        return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #just to fit the notebook
    # image upload
    img = gr.Image(label="Your first image",type="pil")
    upload_btn = gr.Button("Upload")

    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    # upload image
    upload_btn.click(fn=set_image, inputs=[img], outputs=[img])

    btn.click(respond, inputs=[img, msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[img, msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit
gr.close_all()
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a8103959213f1af4ff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


<class 'PIL.Image.Image'>


/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


sunglasses
<class 'PIL.Image.Image'>
The red sneaker is on top of a wave in the ocean.
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
In the image, you can see a red sneaker on top of a wave in the ocean.
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
